In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from random import shuffle

import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 
!mkdir ./logs/

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.callbacks import TensorBoard
# %load_ext tensorboard
# %tensorboard --logdir logs

In [ ]:
base_dir = "../input/dogs-vs-cats-redux-kernels-edition"
train_dir = os.path.join(base_dir, "train.zip")
test_dir = os.path.join(base_dir, "test.zip")

# ../input/dogs-vs-cats-redux-kernels-edition/test.zip

In [ ]:
import zipfile
with zipfile.ZipFile(train_dir,"r") as z:
 z.extractall()

In [ ]:
with zipfile.ZipFile(test_dir,"r") as z:
 z.extractall()

In [ ]:
print(os.listdir("../working/"))


In [ ]:
train_dir  = "../working/train"
test_dir  = "../working/test"

In [ ]:
img_size = 50 
l_rate = 0.001

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(l_rate,'2conv-basic')

In [ ]:
def label_img(img):
    word_label = img.split('.')[-3] # returns cat(-3).1000(-2).jpeg(-1) after split
    if word_label == 'cat':
        return[1]
    elif word_label == 'dog':
        return[0]

In [ ]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(train_dir)):
        label = label_img(img)
        path =  os.path.join(train_dir,img)
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(img_size,img_size))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data

In [ ]:
train
12.5k 50*50   - [[1,0]]


test 
12.5k 50*50    - [1,0,1]

In [ ]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(test_dir)):
        path = os.path.join(test_dir,img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(img_size,img_size))
        testing_data.append([np.array(img),np.array(img_num)])
        
    np.save('test_data.npy',testing_data)
    return testing_data

In [ ]:
train_data =  create_train_data()
### if already saved as numpy array
# train_data =  np.load('train_data.npy')

In [ ]:
for features, labels in train_data[:2]:
    print(features)
    print(labels)

In [ ]:
test_data = process_test_data()

In [ ]:
len(train_data)

In [ ]:
# !ls ../input/training_set/training_set/dogs | wc -l
# ./train
# !ls ../input/training_set/training_set/cats | wc -l
!ls ../working/train | wc -l
!ls ../working/test | wc -l

In [ ]:
# shuffle(train_data)
# for sample in train_data[:10]:
#     print(sample[1])

In [ ]:
print(len(train_data))
print(len(X))
print(len(y))

  

In [ ]:
X= []
y =[]
for features, labels in train_data:
    X.append(features)
    y.append(labels)
X1 = np.array(X).reshape(-1, img_size, img_size, 1)
y1 = np.array(y)#.reshape(-1,1)
print(len(X1))
print(np.array(y).shape)

In [ ]:

    
#4D
# print(X[0].reshape(-1,img_size,img_size,1))
# Try Normalization
# X = X/255.0

X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)#.reshape(-1,1)

# Make X an Array 
# X = np.array([i[0] for i in train_data]).reshape(-1,img_size,img_size,1)
# y =  [i[1]for i in train_data]

#do the same for test_x, test_y
#Try pickle
test_x = []
test_y = []
for features, labels in test_data:
    test_x.append(features)
    test_y.append(labels)
test_x = np.array(test_x).reshape(-1, img_size, img_size, 1)
test_y = np.array(test_y)#.reshape(-1,1)

In [ ]:
np.array(test_y).reshape(-1,1).shape

In [ ]:
test_x.shape[1:]

In [ ]:

print(X.shape,y.shape)

In [ ]:
print(test_y[:5])

In [ ]:
## Checkpoint Practice
# checkpoint_path = 'training_1/cp.ckpt'
checkpoint_path = '../working/logs/Checkpoint{epoch:02d}-{val_accuracy:.02f}.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_dir

##For val_acc, this should be max, for val_loss this should be min
# create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                monitor = 'val_loss',
                                                mode = 'min',
                                                save_weights_only=True,
                                                save_best_only = True,
                                                verbose=1)


In [ ]:
model_clf= Sequential()
# Convolution, Activation,Maxpooling -1
# input shape can also be X.shape[1:]
model_clf.add(Conv2D(32,(3,3),input_shape=(img_size,img_size,1),activation = 'relu'))
model_clf.add(MaxPooling2D(pool_size=(2,2),strides=1)) # Change later
# Convolution, Activation,Maxpooling -2
model_clf.add(Conv2D(64,(3,3),activation = 'relu'))
model_clf.add(MaxPooling2D(pool_size=(2,2),strides=1))
#Flatten
model_clf.add(Flatten())
## FF NN
model_clf.add(Dense(units=128,activation='relu'))
# Sigmoid output probability
model_clf.add(Dense(units=1,activation='sigmoid'))

#DEfine Optimizer 
adam = tf.keras.optimizers.Adam(lr=l_rate)
#Model Compile
# Sigmoid
model_clf.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])

model_clf.save('Cat_vs_Dog-CNN.h5')
#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

In [ ]:
model_clf.summary()

In [ ]:
# Configure the TensorBoard callback and fit your model

tensorboard_callback = TensorBoard("logs")

In [ ]:
print(test_x.shape,test_y.shape)

In [ ]:
1. train = model =YEs
2. test-label  =No
3. validat- label= N0

In [ ]:
#training the model

##1
# history = model_clf.fit(x = training_set, validation_data = testing_set, epochs = 25)
##2
# history = model_clf.fit(X,y,batch_size = 32, validation_split = 0.3, epochs = 25)
##3
history = model_clf.fit(X,y, epochs = 10,
                        callbacks=[cp_callback])